In [56]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [57]:
# analiza sentymentu vader (najpierw sam)
# podzielic tweety po firmie
# aggregacja godzinowo po max, min, mean, median, std, count
# 

In [58]:
tweets = pd.read_csv('company_tweets/apple_tweets.csv')
tweets = tweets.sample(10_000).reset_index(drop=True)

In [59]:
tweets.head()

,tweet_id,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,company_name
0,738794886732734464,2016-06-03 20:10:01,Anticipating #WWDC16: What’s in store for #iCl...,0,0,0,AAPL,apple
1,971617971834703872,2018-03-08 06:25:50,Thursday’s Watch1: $AMZN $ADBE $AAPL $AMD $BTC...,1,2,6,AAPL,apple
2,1055031174123393024,2018-10-24 11:40:08,"Italy’s Antitrust To Fine Samsung, Apple Over ...",0,2,0,AAPL,apple
3,949348512885628928,2018-01-05 19:34:58,Apple App Store revenue grew 34.7% in 2017 $AA...,0,1,1,AAPL,apple
4,911243851880325120,2017-09-22 17:00:39,And who was the goofball telling me yesterday ...,0,0,1,AAPL,apple


In [84]:
def clean_body(col):
    import re
    # usun urle
    # usun hashtagi 
    pattern = r'https?://[a-z.]+/[?a-z0-9./]+'
    col = re.sub(pattern, '', col)
    
    return col

In [85]:
xd = tweets['body'][37]
clean_body(xd)

"Join @RobinhoodApp and we'll both get a share of stock like $AAPL, $F, or $S for free. Make sure to use my link."